In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.0.0'
os.environ['SPARK_VERSION']=spark_version
print(f"http://www.apache.org/dist/spark/{spark_version}/{spark_version}-bin-hadoop2.7.tgz")
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz

!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

http://www.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop2.7.tgz
Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:12 https://developer.download.nvidia.com/comp

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-11-01 02:40:39--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.69MB/s    in 0.6s    

2021-11-01 02:40:40 (1.69 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Home_Improvement_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48881148|R215C9BDXTDQOW|B00FR4YQYK|     381800308|SadoTech Model C ...|Home Improvement|          4|            0|          0|   N|                Y|          Four Stars|        good product| 2015-08-31|
|         US|   47882936|R1DTPUV1J57YHA|B00439MYYE|     921341748|iSpring T32M 3.2 ...|Home Improvement|          5|    

In [5]:
# Create the vine_table. DataFrame
# vine_df = df.select([])
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show(10)
print((vine_df.count(), len(vine_df.columns)))

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R215C9BDXTDQOW|          4|            0|          0|   N|                Y|
|R1DTPUV1J57YHA|          5|            0|          0|   N|                Y|
| RFAZK5EWKJWOU|          5|            0|          0|   N|                Y|
|R2XT8X0O0WS1AL|          5|            0|          0|   N|                Y|
|R14GRNANKO2Y2J|          5|            0|          0|   N|                Y|
|R2BLF9VYL24LCQ|          5|            1|          1|   N|                Y|
|R1GI9UW5KJ671O|          5|            0|          0|   N|                Y|
|R2H5CEJN863M86|          5|            0|          1|   N|                Y|
| R5PPDHFOZ3SMU|          5|            0|          0|   N|                Y|
| RE1L9IENKJJ7Y|          1|            0|          0|   N|     

In [6]:
#products_df.filter(products_df.product_id == "B00E8A0JQS").show(10)
vine20Votes_df = vine_df.filter(vine_df.total_votes >= 20)
vine20Votes_df.show()
print((vine20Votes_df.count(), len(vine20Votes_df.columns)))

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2VIQ3UX794Q0O|          4|           21|         21|   N|                Y|
|R1OA24IIHWF54G|          5|           43|         45|   N|                Y|
| RJ7N3OOJR9RL0|          4|           63|         71|   N|                Y|
|R1W8778CBXSRU6|          5|           28|         29|   N|                Y|
|R2EFAM03SWLIJX|          1|           32|         35|   N|                Y|
|R3F8P56ZDJ6PI8|          4|          106|        115|   N|                Y|
|R2QYLQEK9UAJQ6|          5|           33|         34|   N|                Y|
| RQCC7XNYI014B|          1|           91|         92|   N|                Y|
|R2U8FRHRVW5D2C|          5|           23|         23|   N|                N|
|R3RJZC5J4VP7AT|          4|           86|         91|   N|     

In [7]:
vinePercentage_df = vine20Votes_df.filter((vine20Votes_df.helpful_votes/vine20Votes_df.total_votes) >= .5)
vinePercentage_df.show()
print((vinePercentage_df.count(), len(vinePercentage_df.columns)))

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2VIQ3UX794Q0O|          4|           21|         21|   N|                Y|
|R1OA24IIHWF54G|          5|           43|         45|   N|                Y|
| RJ7N3OOJR9RL0|          4|           63|         71|   N|                Y|
|R1W8778CBXSRU6|          5|           28|         29|   N|                Y|
|R2EFAM03SWLIJX|          1|           32|         35|   N|                Y|
|R3F8P56ZDJ6PI8|          4|          106|        115|   N|                Y|
|R2QYLQEK9UAJQ6|          5|           33|         34|   N|                Y|
| RQCC7XNYI014B|          1|           91|         92|   N|                Y|
|R2U8FRHRVW5D2C|          5|           23|         23|   N|                N|
|R3RJZC5J4VP7AT|          4|           86|         91|   N|     

In [8]:
vinePaid_df = vinePercentage_df.filter(vinePercentage_df.vine == 'Y')
vinePaid_df.show()
print((vinePaid_df.count(), len(vinePaid_df.columns)))

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R29V4UDSS053D8|          3|           33|         38|   Y|                N|
|R1I2D068WC37PA|          3|           32|         37|   Y|                N|
|R13W2U74F67QED|          5|           62|         76|   Y|                N|
|R2QI37XFOBKUGD|          4|           24|         33|   Y|                N|
|R19F60BB2DNCKN|          5|          170|        174|   Y|                N|
|R2PYNEWYYQDWHI|          5|           24|         27|   Y|                N|
|R1UEWHCPJL2XJB|          5|           70|         72|   Y|                N|
| RXTMMEBRYZB53|          5|           21|         24|   Y|                N|
|R1PXVYIYMYGUZL|          5|           21|         26|   Y|                N|
| RSFMJJXFZHX1C|          5|           24|         25|   Y|     

In [9]:
vineUnpaid_df = vinePercentage_df.filter(vinePercentage_df.vine == 'N')
vineUnpaid_df.show()
print((vineUnpaid_df.count(), len(vineUnpaid_df.columns)))

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2VIQ3UX794Q0O|          4|           21|         21|   N|                Y|
|R1OA24IIHWF54G|          5|           43|         45|   N|                Y|
| RJ7N3OOJR9RL0|          4|           63|         71|   N|                Y|
|R1W8778CBXSRU6|          5|           28|         29|   N|                Y|
|R2EFAM03SWLIJX|          1|           32|         35|   N|                Y|
|R3F8P56ZDJ6PI8|          4|          106|        115|   N|                Y|
|R2QYLQEK9UAJQ6|          5|           33|         34|   N|                Y|
| RQCC7XNYI014B|          1|           91|         92|   N|                Y|
|R2U8FRHRVW5D2C|          5|           23|         23|   N|                N|
|R3RJZC5J4VP7AT|          4|           86|         91|   N|     

In [10]:
totalReviews = vinePaid_df.count()
fiveStarReviews = vinePaid_df.filter(vinePaid_df.star_rating == 5).count()
percentageFiveStarReviews = ((fiveStarReviews*100)/totalReviews)

In [11]:
print('----------------PAID VINES -----------------')
print('Total reviews count: '+str(totalReviews))
print('Five Star reviews count: '+str(fiveStarReviews))
print('Percentage: '+str(percentageFiveStarReviews))

----------------PAID VINES -----------------
Total reviews count: 266
Five Star reviews count: 125
Percentage: 46.99248120300752


In [12]:
totalReviewsU = vineUnpaid_df.count()
fiveStarReviewsU = vineUnpaid_df.filter(vineUnpaid_df.star_rating == 5).count()
percentageFiveStarReviewsU = ((fiveStarReviewsU*100)/totalReviewsU)

In [13]:
print('----------------UNPAID VINES -----------------')
print('Total reviews count: '+str(totalReviewsU))
print('Five Star reviews count: '+str(fiveStarReviewsU))
print('Percentage: '+str(percentageFiveStarReviewsU))

----------------UNPAID VINES -----------------
Total reviews count: 38829
Five Star reviews count: 18246
Percentage: 46.99065131731438
